In [ ]:
import numpy as np
from matplotlib import image
import matplotlib.pyplot as plt

In [ ]:
def read_image(filename):
    img = image.imread(filename)
    return img

In [ ]:
def show_image(img, colormap = ''):
    plt.imshow(img)
    plt.show()

In [ ]:
img = read_image('imagens/peppers.bmp')
show_image(img, 'hot')